In [1]:
!lscpu

Architecture:             x86_64
  CPU op-mode(s):         32-bit, 64-bit
  Address sizes:          46 bits physical, 48 bits virtual
  Byte Order:             Little Endian
CPU(s):                   8
  On-line CPU(s) list:    0-7
Vendor ID:                GenuineIntel
  Model name:             Intel(R) Xeon(R) CPU @ 2.20GHz
    CPU family:           6
    Model:                79
    Thread(s) per core:   2
    Core(s) per socket:   4
    Socket(s):            1
    Stepping:             0
    BogoMIPS:             4399.99
    Flags:                fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge m
                          ca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht sysc
                          all nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xt
                          opology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq
                           ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt
                           aes xsave avx f16c rdrand hypervisor 

**-----------Load The Data------------**

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import time
from collections import deque
from tqdm import tqdm
from typing import Dict, List, Tuple
import math
import random
import numpy as np
from numpy import ndarray

In [4]:
# Set seeds for reproducibility
# --- Fixed parameters ---
dim = 9 # dimension of the points
np.random.seed(42)  # set seed for reproducibility

In [5]:
# Define the dataset path
dataset_path = '/content/drive/MyDrive/datasets/dataSearch/CO2_QM_ds_mod_augmented.txt'

# Load the dataset
# If the file has a different delimiter (e.g., commas), you can set delimiter=',' accordingly.
dataset = np.loadtxt(dataset_path)
total_points = len(dataset)

In [6]:
# Data sizes:
SB = 10_000    # Small Batch: 10K points
MB = 100_000   # Medium Batch: 100K points
LB = 1_000_000 # Large Batch: 1M points

# Query sizes:
SQ = 20        # Small Query: 20 queries
MQ = 100       # Medium Query: 100 queries
LQ = 500       # Large Query: 500 queries

# === Define the test combinations ===
# Each tuple is of the form (data_size, query_size)
test_combinations = [
    (LB, SQ)
]

In [16]:
# ----------------------------
# Phase 1: Pre-generate and Store Queries from the Dataset
# ----------------------------

queries_list = []
st_idx = 0  # Starting index for slicing the dataset

for data_size, query_size in test_combinations:
    # Slice the current data chunk
    end_idx = st_idx + data_size
    current_data = dataset[st_idx:end_idx]
    st_idx = end_idx  # Update start index for next chunk

    # Ensure query selection is different for each batch size
    if query_size <= len(current_data):  # Sampling from the current batch only
        random_indices = np.random.choice(len(current_data), size=query_size, replace=False)
        queries = current_data[random_indices]  # Select query points from the current batch
    else:
        raise ValueError(f"Query size {query_size} exceeds available data points {len(current_data)}.")

    # Append the selected queries to the list
    queries_list.append(queries)


----------------------------

**--------QuickSelect Algorithm----------------**

In [8]:
from collections import deque
from typing import List, Tuple

#*-----------------------------------------------------------------------------
def quickselect(distances:List, k:int):
    """QuickSelect algorithm to find the k-th smallest distance."""

    if len(distances) == 1:
        return distances[0]

    if len(distances) == 0:
        return None

    stack = deque()
    stack.append( (distances, k) )
    while stack:

        distances, k = stack.pop()
        if len(distances) == 1:
            return distances[0]

        # Returned as a tuple
        pivot = median_of_medians(distances)
        lows, highs, pivots = partition(distances, pivot)

        if len(lows) > k:
            stack.append((lows, k))
        elif len(lows) + len(pivots) > k:
            return pivot
        else:
            stack.append((highs, k - len(lows) - len(pivots)))
    return pivot

#*-----------------------------------------------------------------------------
def partition(distances, pivot):
    lows, highs, pivots = [], [], []

    for d in distances:
        if d[0] < pivot[0]:
            lows.append(d)
        elif d[0] > pivot[0]:
            highs.append(d)
        else:
            pivots.append(d)
    return lows, highs, pivots

#*-----------------------------------------------------------------------------
def median_of_medians(distances, grp_size=5):
    """Find the index of the median distance in a list of distances.
    Returns the index and the median distance as a tuple."""

    stack = deque()
    # Split the list in chunks of grp_size
    # And add the lists to the stack on the right side
    for i in range(0, len(distances), grp_size):
        subList = distances[i:i+grp_size]
        insertion_sort(subList)
        stack.append(subList)

    # We want to find the median of the medians
    # and return the index of that median
    median_list = []
    stack_count = len(stack)
    while 0 < stack_count:

        subList = stack.pop()
        stack_count -= 1

        mid_index = len(subList) // 2
        median_list.append(subList[mid_index])

        if len(median_list) == grp_size:
            insertion_sort(median_list)
            # append to the left, since we are popping from the right
            stack.appendleft(median_list)
            if stack_count == 0:
                stack_count = len(stack)
            median_list = []
        # If the count is 0, we have reached the end of the old stack
        # and need to insert the remaining elements,
        # stack count and the median list will be reset
        if stack_count == 0 and len(median_list) > 0:
            if len(median_list) > 1:
                insertion_sort(median_list)
            stack.appendleft(median_list)
            stack_count = len(stack)
            if stack_count == 1 and len(median_list) == 1:
                return median_list[0]
            median_list = []

#*-----------------------------------------------------------------------------
def insertion_sort(arr):
# In-place insertion sort, efficient for small arrays
    for i in range(1, len(arr)):
        key = arr[i]
        j = i - 1
        while j >= 0 and arr[j][0] > key[0]:
            arr[j + 1] = arr[j]
            j -= 1
        arr[j + 1] = key
#*-----------------------------------------------------------------------------

----------------------------

**----------KD Tree Hybrid---------------------**

In [11]:
import numpy as np
from collections import deque
from typing import List, Tuple, Union

#*-----------------------------------------------------------------------------
class KDNode:
    def __init__(self, points=None, left=None, right=None, parent=None):
        self.points = points if points is not None else []  # Now always a list of points
        self.split_dim = None
        self.split_val = None
        self.leaf = False
        self.left_tr = left
        self.right_tr = right
        self.parent = parent
        self.centroid = None
        self.radius = 0
        # NumPy array cache for search operations
        self._np_points = None

#*-----------------------------------------------------------------------------
class KDTree:
    def __init__(self, leaf_size=40, epsilon=0.1):
        self.points = None
        self.root = None
        self.dim = None
        self.leaf_size = leaf_size
        self.epsilon = epsilon

    #*-----------------------------------------------------------------------------
    def find_best_split_dimension(self, points: np.ndarray, start_dim: int) -> Tuple[int, bool]:
        if len(points) == 0:
            return 0, True
        for d in range(self.dim):
            curr_dim = (start_dim + d) % self.dim
            if not np.all(points[:, curr_dim] == points[0, curr_dim]):
                return curr_dim, False
        return 0, True

    #*-----------------------------------------------------------------------------
    def build(self, points=None):
        self.points = points
        self.dim = points.shape[1]
        self.root = KDNode()
        stack = deque([(self.root, points, 0)])

        while stack:
            node, node_points, depth = stack.pop()

            if len(node_points) <= self.leaf_size:
                node.leaf = True
                node.points = node_points.tolist() # Convert to list of points
                node.centroid = np.mean(node_points, axis=0)
                if len(node_points) > 1:
                    node.radius = np.max(np.linalg.norm(node_points - node.centroid, axis=1))
                continue

            split_dim, all_equal = self.find_best_split_dimension(node_points, depth % self.dim)
            if all_equal:
                node.leaf = True
                node.points = node_points.tolist()
                node.centroid = np.mean(node_points, axis=0)
                if len(node_points) > 1:
                    node.radius = np.max(np.linalg.norm(node_points - node.centroid, axis=1))
                continue

            node.split_dim = split_dim
            coords = [(p[split_dim], i) for i, p in enumerate(node_points)]
            k = len(node_points) // 2
            med_ = quickselect(coords, k)
            median_val = med_[0]
            node.split_val = median_val

            left_points = node_points[node_points[:, split_dim] < (median_val - self.epsilon)]
            right_points = node_points[node_points[:, split_dim] > (median_val + self.epsilon)]
            middle_points = node_points[
                (node_points[:, split_dim] >= (median_val - self.epsilon)) &
                (node_points[:, split_dim] <= (median_val + self.epsilon))
            ]

            node.points = [p for p in middle_points]
            if len(middle_points) > 0:
                node.centroid = np.mean(middle_points, axis=0)
                if len(middle_points) > 1:
                    node.radius = np.max(np.linalg.norm(middle_points - node.centroid, axis=1))

            if len(left_points) > 0:
                node.left_tr = KDNode(parent=node)
                stack.append((node.left_tr, left_points, depth + 1))
            if len(right_points) > 0:
                node.right_tr = KDNode(parent=node)
                stack.append((node.right_tr, right_points, depth + 1))
            if len(left_points) == 0 and len(right_points) == 0:
                node.leaf = True

        return self.root

    #*-----------------------------------------------------------------------------
    def insert(self, point: Union[List, np.ndarray]):
        if self.root is None:
            self.dim = len(point)
            self.points = [point]
            self.root = KDNode()
            self.root.points = [point]
            self.root.centroid = point
            self.root.radius = 0
            self.root.leaf = True
            return

        current = self.root

        while not current.leaf:
            split_dim = current.split_dim
            split_val = current.split_val

            if (len(current.points) > 0 and len(current.points) < self.leaf_size
                and abs(point[split_dim] - split_val) <= self.epsilon):
                n = len(current.points)
                current.points.append(point)
                current.centroid = (n * current.centroid + point) / (n + 1)
                dist_to_centroid = np.linalg.norm(point - current.centroid)
                current.radius = max(current.radius, dist_to_centroid)
                # Clear the NumPy cache since we modified the points
                current._np_points = None
                return

            if point[split_dim] < split_val - self.epsilon:
                if current.left_tr is None:
                    current.left_tr = KDNode(parent=current)
                    current.left_tr.points = [point]
                    current.left_tr.centroid = point
                    current.left_tr.radius = 0
                    current.left_tr.leaf = True
                    return
                current = current.left_tr
            else:
                if current.right_tr is None:
                    current.right_tr = KDNode(parent=current)
                    current.right_tr.points = [point]
                    current.right_tr.centroid = point
                    current.right_tr.radius = 0
                    current.right_tr.leaf = True
                    return
                current = current.right_tr

        n = len(current.points)

        current.points.append(point)
        current.centroid = (n * current.centroid + point) / (n + 1)
        dist_to_centroid = np.linalg.norm(point - current.centroid)
        current.radius = max(current.radius, dist_to_centroid)
        # Clear the NumPy cache since we modified the points
        current._np_points = None

    #*-----------------------------------------------------------------------------
    def prepare_for_search(self):
        """
        Convert all list-based point collections to numpy arrays for efficient search.
        This should be called once after all insertions and before searches.
        """

        stack = [self.root]

        while stack:
            node = stack.pop()

            if node is None:
                continue

            # Convert points to numpy array
            if node.points:
                node._np_points = np.array(node.points)

            # Add child nodes to stack
            if node.left_tr:
                stack.append(node.left_tr)
            if node.right_tr:
                stack.append(node.right_tr)

    #*-----------------------------------------------------------------------------
    def search(self, query):
        """
        Search for the nearest neighbor of the query point using KD-tree.
        Uses triangle inequality pruning at leaf nodes.

        Parameters:
            query (np.ndarray): The query point

        Returns:
            Tuple of (distance to nearest neighbor, nearest neighbor point)
        """

        best_dist = float('inf')
        best_point = None

        # Stack for depth-first traversal
        stack = [self.root]

        while stack:
            node = stack.pop()

            if node.leaf:
                dist_to_centroid = np.linalg.norm(query - node.centroid)
                if dist_to_centroid - node.radius >= best_dist:
                    continue

                # Check if _np_points exists, if not, create it
                if node._np_points is None and node.points:
                    node._np_points = np.array(node.points)

                # Use the cached NumPy array for distance calculations
                if node._np_points is not None:
                    dists = np.linalg.norm(node._np_points - query, axis=1)
                    min_idx = np.argmin(dists)
                    if dists[min_idx] < best_dist:
                        best_dist = dists[min_idx]
                        best_point = node._np_points[min_idx]
                continue

            if node.split_dim is not None:
                if len(node.points) > 0:
                    # Check if _np_points exists, if not, create it
                    if node._np_points is None and node.points:
                        node._np_points = np.array(node.points)

                    # Use the cached NumPy array for distance calculations
                    if node._np_points is not None:
                        dists = np.linalg.norm(node._np_points - query, axis=1)
                        min_idx = np.argmin(dists)
                        if dists[min_idx] < best_dist:
                            best_dist = dists[min_idx]
                            best_point = node._np_points[min_idx]

                split_dim = node.split_dim
                split_val = node.split_val
                diff = query[split_dim] - split_val

                if diff < -self.epsilon:
                    if node.right_tr and abs(diff) < best_dist:
                        stack.append(node.right_tr)
                    if node.left_tr:
                        stack.append(node.left_tr)
                elif diff > self.epsilon:
                    if node.left_tr and abs(diff) < best_dist:
                        stack.append(node.left_tr)
                    if node.right_tr:
                        stack.append(node.right_tr)
                else:
                    if abs(diff) <= self.epsilon:
                        if diff < 0:
                            if node.left_tr:
                                stack.append(node.left_tr)
                            if node.right_tr:
                                stack.append(node.right_tr)
                        else:
                            if node.right_tr:
                                stack.append(node.right_tr)
                            if node.left_tr:
                                stack.append(node.left_tr)

        return best_dist, best_point
#*-----------------------------------------------------------------------------

-------------------------

**-------KD Tree Node Size Optimization For Rebuild-------**

**--------------Run 1---------------------**

In [ ]:
import time
from tqdm import tqdm

# Test code for KD Tree
results = []

# Define leaf size range
leaf_size_range_start = 50
leaf_size_range_end = 2_500  # Ensure inclusive range
leaf_size_step = 50

# Fix the epsilon at 0.0
epsilon=0.0

data_size, query_size = test_combinations[0]

# Obtain Dataset Sample
current_data = dataset[:data_size]  # Directly slicing for clarity

# Retrieve the pre-generated queries from the list using the same order
queries = queries_list[0]  # Use first query set, as per your specification

for idx, leaf_size in enumerate(range(leaf_size_range_start, leaf_size_range_end + leaf_size_step, leaf_size_step)):
    print(f"\n{idx+1}: === Testing with Leaf Size: {leaf_size:,}, Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    # --- Build the KD Tree ---
    build_start = time.perf_counter()
    kdtree = KDTree(leaf_size=leaf_size, epsilon=epsilon)
    kdtree.build(current_data)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying (Leaf Size {leaf_size:,})"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    search_time = search_end - search_start

    total_time = build_time + search_time
    results.append((leaf_size, build_time, search_time, total_time))

    print("-------------------------------------------------------------")
    print(f"Build Time: {build_time:.4f} sec, Search Time: {search_time:.4f} sec")
    print(f"Leaf Size: {leaf_size:,}, Total Time: {total_time:.4f} sec")

print("\n=== Summary of KD Tree Node Size Test ===")
for l_size, b_time, s_time, t_time in results:
    print(f"Leaf Size: {l_size:,}, Build Time: {b_time:.4f} sec, Search Time: {s_time:.4f} sec, Total Time: {t_time:.4f} sec")



1: === Testing with Leaf Size: 50, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Leaf Size 50): 100%|██████████| 20/20 [00:00<00:00, 920.82it/s]


-------------------------------------------------------------
Build Time: 23.4508 sec, Search Time: 0.5595 sec
Leaf Size: 50, Total Time: 24.0103 sec

2: === Testing with Leaf Size: 100, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Leaf Size 100): 100%|██████████| 20/20 [00:00<00:00, 1041.25it/s]


-------------------------------------------------------------
Build Time: 21.9471 sec, Search Time: 0.5107 sec
Leaf Size: 100, Total Time: 22.4577 sec

3: === Testing with Leaf Size: 150, Data Size: 1,000,000 points, Query Size: 20 queries ===


...

-------------------------------------------------------------
Build Time: 14.6408 sec, Search Time: 0.4803 sec
Leaf Size: 2,400, Total Time: 15.1211 sec

49: === Testing with Leaf Size: 2,450, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Leaf Size 2,450): 100%|██████████| 20/20 [00:00<00:00, 837.35it/s]


-------------------------------------------------------------
Build Time: 14.5649 sec, Search Time: 0.4724 sec
Leaf Size: 2,450, Total Time: 15.0373 sec

50: === Testing with Leaf Size: 2,500, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Leaf Size 2,500): 100%|██████████| 20/20 [00:00<00:00, 815.77it/s]

-------------------------------------------------------------
Build Time: 14.4714 sec, Search Time: 0.4808 sec
Leaf Size: 2,500, Total Time: 14.9521 sec

=== Summary of KD Tree Node Size Test ===
Leaf Size: 50, Build Time: 23.4508 sec, Search Time: 0.5595 sec, Total Time: 24.0103 sec
Leaf Size: 100, Build Time: 21.9471 sec, Search Time: 0.5107 sec, Total Time: 22.4577 sec
Leaf Size: 150, Build Time: 19.9047 sec, Search Time: 0.4903 sec, Total Time: 20.3950 sec
Leaf Size: 200, Build Time: 19.8730 sec, Search Time: 0.4929 sec, Total Time: 20.3658 sec
Leaf Size: 250, Build Time: 18.2483 sec, Search Time: 0.4746 sec, Total Time: 18.7229 sec
Leaf Size: 300, Build Time: 18.1920 sec, Search Time: 0.4769 sec, Total Time: 18.6689 sec
Leaf Size: 350, Build Time: 18.3562 sec, Search Time: 0.4846 sec, Total Time: 18.8408 sec
Leaf Size: 400, Build Time: 18.4243 sec, Search Time: 0.4737 sec, Total Time: 18.8980 sec
Leaf Size: 450, Build Time: 18.3295 sec, Search Time: 0.4768 sec, Total Time: 18.8063

**-------------Run 2-----------------------**

In [ ]:
import time
from tqdm import tqdm

# Test code for KD Tree
results = []

# Define leaf size range
leaf_size_range_start = 50
leaf_size_range_end = 2_500  # Ensure inclusive range
leaf_size_step = 50

# Fix the epsilon at 0.0
epsilon=0.0

data_size, query_size = test_combinations[0]

# Obtain Dataset Sample
current_data = dataset[:data_size]  # Directly slicing for clarity

# Retrieve the pre-generated queries from the list using the same order
queries = queries_list[0]  # Use first query set, as per your specification

for idx, leaf_size in enumerate(range(leaf_size_range_start, leaf_size_range_end + leaf_size_step, leaf_size_step)):
    print(f"\n{idx+1}: === Testing with Leaf Size: {leaf_size:,}, Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    # --- Build the KD Tree ---
    build_start = time.perf_counter()
    kdtree = KDTree(leaf_size=leaf_size, epsilon=epsilon)
    kdtree.build(current_data)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying (Leaf Size {leaf_size:,})"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    search_time = search_end - search_start

    total_time = build_time + search_time
    results.append((leaf_size, build_time, search_time, total_time))

    print("-------------------------------------------------------------")
    print(f"Build Time: {build_time:.4f} sec, Search Time: {search_time:.4f} sec")
    print(f"Leaf Size: {leaf_size:,}, Total Time: {total_time:.4f} sec")

print("\n=== Summary of KD Tree Node Size Test ===")
for l_size, b_time, s_time, t_time in results:
    print(f"Leaf Size: {l_size:,}, Build Time: {b_time:.4f} sec, Search Time: {s_time:.4f} sec, Total Time: {t_time:.4f} sec")



1: === Testing with Leaf Size: 50, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Leaf Size 50): 100%|██████████| 20/20 [00:00<00:00, 907.57it/s]


-------------------------------------------------------------
Build Time: 23.6886 sec, Search Time: 0.5212 sec
Leaf Size: 50, Total Time: 24.2097 sec

2: === Testing with Leaf Size: 100, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Leaf Size 100): 100%|██████████| 20/20 [00:00<00:00, 1049.38it/s]


-------------------------------------------------------------
Build Time: 21.7819 sec, Search Time: 0.5046 sec
Leaf Size: 100, Total Time: 22.2865 sec

3: === Testing with Leaf Size: 150, Data Size: 1,000,000 points, Query Size: 20 queries ===


...

-------------------------------------------------------------
Build Time: 14.9791 sec, Search Time: 0.4714 sec
Leaf Size: 2,400, Total Time: 15.4505 sec

49: === Testing with Leaf Size: 2,450, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Leaf Size 2,450): 100%|██████████| 20/20 [00:00<00:00, 864.79it/s]


-------------------------------------------------------------
Build Time: 14.6858 sec, Search Time: 0.4758 sec
Leaf Size: 2,450, Total Time: 15.1617 sec

50: === Testing with Leaf Size: 2,500, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Leaf Size 2,500): 100%|██████████| 20/20 [00:00<00:00, 853.98it/s]

-------------------------------------------------------------
Build Time: 15.1009 sec, Search Time: 0.4720 sec
Leaf Size: 2,500, Total Time: 15.5729 sec

=== Summary of KD Tree Node Size Test ===
Leaf Size: 50, Build Time: 23.6886 sec, Search Time: 0.5212 sec, Total Time: 24.2097 sec
Leaf Size: 100, Build Time: 21.7819 sec, Search Time: 0.5046 sec, Total Time: 22.2865 sec
Leaf Size: 150, Build Time: 20.1834 sec, Search Time: 0.4877 sec, Total Time: 20.6711 sec
Leaf Size: 200, Build Time: 19.5789 sec, Search Time: 0.4928 sec, Total Time: 20.0717 sec
Leaf Size: 250, Build Time: 18.4355 sec, Search Time: 0.4727 sec, Total Time: 18.9081 sec
Leaf Size: 300, Build Time: 18.4102 sec, Search Time: 0.4734 sec, Total Time: 18.8837 sec
Leaf Size: 350, Build Time: 18.0325 sec, Search Time: 0.4759 sec, Total Time: 18.5084 sec
Leaf Size: 400, Build Time: 18.4032 sec, Search Time: 0.4775 sec, Total Time: 18.8806 sec
Leaf Size: 450, Build Time: 18.1963 sec, Search Time: 0.4743 sec, Total Time: 18.6706

**-------------Opt Node Size For Rebuild-------------**

In [ ]:
import numpy as np

# Leaf sizes
leaf_sizes = np.array(
    [  50, 100, 150, 200, 250, 300, 350, 400, 450, 500, 550, 600, 650, 700,
  750, 800, 850, 900, 950,1000,1050,1100,1150,1200,1250,1300,1350,1400,
 1450,1500,1550,1600,1650,1700,1750,1800,1850,1900,1950,2000,2050,2100,
 2150,2200,2250,2300,2350,2400,2450,2500]
    )

# Total times from two runs (or versions)
total_times_1 = np.array(
    [24.0103,22.4577,20.395 ,20.3658,18.7229,18.6689,18.8408,18.898 ,18.8063,
 17.5287,17.7908,17.4828,17.4023,17.7092,17.3727,17.9302,17.5698,17.5688,
 17.4662,16.138 ,16.2971,16.2477,16.1719,16.1217,16.3335,16.205 ,16.3942,
 16.137 ,16.2367,16.1896,16.2607,16.3187,16.1057,16.0831,16.1596,16.1232,
 16.316 ,15.9498,15.557 ,15.0822,14.8262,14.9616,14.9551,15.0288,15.0487,
 14.8209,14.9721,15.1211,15.0373,14.9521]
    )

total_times_2 = np.array(
    [24.2097,22.2865,20.6711,20.0717,18.9081,18.8837,18.5084,18.8806,18.6706,
 17.6501,17.9964,17.6599,17.7756,17.5571,17.6493,17.6242,17.7416,17.7566,
 17.3478,16.5101,16.5551,16.5682,16.4598,16.5466,16.5656,16.4776,16.5615,
 16.5704,16.8796,16.7463,16.6555,16.478 ,16.5255,16.4438,16.3386,16.3104,
 16.3155,16.3398,16.0048,15.3651,15.0016,15.3697,15.1363,15.1452,15.0828,
 15.3096,15.2131,15.4505,15.1617,15.5729]
    )

# Average total time across the two runs
avg_total_times = (total_times_1 + total_times_2) / 2

# Index of the minimum average total time
min_idx = np.argmin(avg_total_times)

# Corresponding leaf size
optimal_leaf_size = leaf_sizes[min_idx]

# Print results
print(f"Average total time (sec): {avg_total_times[min_idx]:.4f}")
print(f"Minimum found at leaf size: {optimal_leaf_size}")


Average total time (sec): 14.9139
Minimum found at leaf size: 2050


**-----Epsilon Optimization For Rebuild------**

**-----------Run 1-------------------**

In [ ]:
import time
from tqdm import tqdm

# Test code for KD Tree
results = []

leaf_size = 2050
# Define epsilon range
epsilon_range_start = 0.15
epsilon_range_end = 0.25  # Ensure inclusive range
epsilon_size_step = 0.005

data_size, query_size = test_combinations[0]

# Obtain Dataset Sample
current_data = dataset[:data_size]  # Directly slicing for clarity

# Retrieve the pre-generated queries from the list using the same order
queries = queries_list[0]  # Use first query set, as per your specification

for idx, epsilon in enumerate(np.arange(epsilon_range_start, epsilon_range_end + epsilon_size_step, epsilon_size_step)):
    epsilon = round(epsilon, 10)
    print(f"\n{idx+1}: === Testing with Epsilon: {epsilon:,}, Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    # --- Build the KD Tree ---
    build_start = time.perf_counter()
    kdtree = KDTree(leaf_size=leaf_size, epsilon=epsilon)
    kdtree.build(current_data)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying (Leaf Size {leaf_size:,})"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    search_time = search_end - search_start

    total_time = build_time + search_time
    results.append((epsilon, build_time, search_time, total_time))

    print("-------------------------------------------------------------")
    print(f"Build Time: {build_time:.4f} sec, Search Time: {search_time:.4f} sec")
    print(f"Epsilon: {epsilon:,}, Total Time: {total_time:.4f} sec")

print("\n=== Summary of KD Tree Epsilon Size Test ===")
for e_size, b_time, s_time, t_time in results:
    print(f"Epsilon: {e_size:,}, Build Time: {b_time:.4f} sec, Search Time: {s_time:.4f} sec, Total Time: {t_time:.4f} sec")



1: === Testing with Epsilon: 0.15, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Leaf Size 2,050): 100%|██████████| 20/20 [00:01<00:00, 19.83it/s]


-------------------------------------------------------------
Build Time: 2.8703 sec, Search Time: 1.1860 sec
Epsilon: 0.15, Total Time: 4.0563 sec

2: === Testing with Epsilon: 0.155, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Leaf Size 2,050): 100%|██████████| 20/20 [00:01<00:00, 19.60it/s]


-------------------------------------------------------------
Build Time: 2.7731 sec, Search Time: 1.2006 sec
Epsilon: 0.155, Total Time: 3.9737 sec

3: === Testing with Epsilon: 0.16, Data Size: 1,000,000 points, Query Size: 20 queries ===


...

-------------------------------------------------------------
Build Time: 2.2317 sec, Search Time: 1.2622 sec
Epsilon: 0.24, Total Time: 3.4939 sec

20: === Testing with Epsilon: 0.245, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Leaf Size 2,050): 100%|██████████| 20/20 [00:01<00:00, 18.47it/s]


-------------------------------------------------------------
Build Time: 2.3234 sec, Search Time: 1.2680 sec
Epsilon: 0.245, Total Time: 3.5914 sec

21: === Testing with Epsilon: 0.25, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Leaf Size 2,050): 100%|██████████| 20/20 [00:01<00:00, 18.02it/s]

-------------------------------------------------------------
Build Time: 2.2958 sec, Search Time: 1.3056 sec
Epsilon: 0.25, Total Time: 3.6014 sec

=== Summary of KD Tree Epsilon Size Test ===
Epsilon: 0.15, Build Time: 2.8703 sec, Search Time: 1.1860 sec, Total Time: 4.0563 sec
Epsilon: 0.155, Build Time: 2.7731 sec, Search Time: 1.2006 sec, Total Time: 3.9737 sec
Epsilon: 0.16, Build Time: 2.6508 sec, Search Time: 1.1991 sec, Total Time: 3.8499 sec
Epsilon: 0.165, Build Time: 2.5702 sec, Search Time: 1.2070 sec, Total Time: 3.7772 sec
Epsilon: 0.17, Build Time: 2.6287 sec, Search Time: 1.2207 sec, Total Time: 3.8494 sec
Epsilon: 0.175, Build Time: 2.4802 sec, Search Time: 1.2079 sec, Total Time: 3.6880 sec
Epsilon: 0.18, Build Time: 2.5158 sec, Search Time: 1.2072 sec, Total Time: 3.7230 sec
Epsilon: 0.185, Build Time: 2.5437 sec, Search Time: 1.2350 sec, Total Time: 3.7787 sec
Epsilon: 0.19, Build Time: 2.4224 sec, Search Time: 1.2271 sec, Total Time: 3.6495 sec
Epsilon: 0.195, Bui

**----------Run 2--------------------**

In [ ]:
import time
from tqdm import tqdm

# Test code for KD Tree
results = []

leaf_size = 2050
# Define epsilon range
epsilon_range_start = 0.15
epsilon_range_end = 0.25  # Ensure inclusive range
epsilon_size_step = 0.005

data_size, query_size = test_combinations[0]

# Obtain Dataset Sample
current_data = dataset[:data_size]  # Directly slicing for clarity

# Retrieve the pre-generated queries from the list using the same order
queries = queries_list[0]  # Use first query set, as per your specification

for idx, epsilon in enumerate(np.arange(epsilon_range_start, epsilon_range_end + epsilon_size_step, epsilon_size_step)):
    epsilon = round(epsilon, 10)
    print(f"\n{idx+1}: === Testing with Epsilon: {epsilon:,}, Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    # --- Build the KD Tree ---
    build_start = time.perf_counter()
    kdtree = KDTree(leaf_size=leaf_size, epsilon=epsilon)
    kdtree.build(current_data)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying (Leaf Size {leaf_size:,})"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    search_time = search_end - search_start

    total_time = build_time + search_time
    results.append((epsilon, build_time, search_time, total_time))

    print("-------------------------------------------------------------")
    print(f"Build Time: {build_time:.4f} sec, Search Time: {search_time:.4f} sec")
    print(f"Epsilon: {epsilon:,}, Total Time: {total_time:.4f} sec")

print("\n=== Summary of KD Tree Epsilon Size Test ===")
for e_size, b_time, s_time, t_time in results:
    print(f"Epsilon: {e_size:,}, Build Time: {b_time:.4f} sec, Search Time: {s_time:.4f} sec, Total Time: {t_time:.4f} sec")



1: === Testing with Epsilon: 0.15, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Leaf Size 2,050): 100%|██████████| 20/20 [00:01<00:00, 19.61it/s]


-------------------------------------------------------------
Build Time: 2.7158 sec, Search Time: 1.1974 sec
Epsilon: 0.15, Total Time: 3.9131 sec

2: === Testing with Epsilon: 0.155, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Leaf Size 2,050): 100%|██████████| 20/20 [00:01<00:00, 19.34it/s]


-------------------------------------------------------------
Build Time: 2.7001 sec, Search Time: 1.2109 sec
Epsilon: 0.155, Total Time: 3.9110 sec

3: === Testing with Epsilon: 0.16, Data Size: 1,000,000 points, Query Size: 20 queries ===


...

-------------------------------------------------------------
Build Time: 2.3133 sec, Search Time: 1.2762 sec
Epsilon: 0.24, Total Time: 3.5895 sec

20: === Testing with Epsilon: 0.245, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Leaf Size 2,050): 100%|██████████| 20/20 [00:01<00:00, 18.20it/s]


-------------------------------------------------------------
Build Time: 2.2262 sec, Search Time: 1.2763 sec
Epsilon: 0.245, Total Time: 3.5025 sec

21: === Testing with Epsilon: 0.25, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Leaf Size 2,050): 100%|██████████| 20/20 [00:01<00:00, 18.42it/s]

-------------------------------------------------------------
Build Time: 2.2465 sec, Search Time: 1.2673 sec
Epsilon: 0.25, Total Time: 3.5138 sec

=== Summary of KD Tree Epsilon Size Test ===
Epsilon: 0.15, Build Time: 2.7158 sec, Search Time: 1.1974 sec, Total Time: 3.9131 sec
Epsilon: 0.155, Build Time: 2.7001 sec, Search Time: 1.2109 sec, Total Time: 3.9110 sec
Epsilon: 0.16, Build Time: 2.5931 sec, Search Time: 1.2069 sec, Total Time: 3.8000 sec
Epsilon: 0.165, Build Time: 2.6299 sec, Search Time: 1.2232 sec, Total Time: 3.8531 sec
Epsilon: 0.17, Build Time: 2.5985 sec, Search Time: 1.2192 sec, Total Time: 3.8177 sec
Epsilon: 0.175, Build Time: 2.6205 sec, Search Time: 1.2634 sec, Total Time: 3.8840 sec
Epsilon: 0.18, Build Time: 2.5593 sec, Search Time: 1.2279 sec, Total Time: 3.7872 sec
Epsilon: 0.185, Build Time: 2.4483 sec, Search Time: 1.2352 sec, Total Time: 3.6835 sec
Epsilon: 0.19, Build Time: 2.4360 sec, Search Time: 1.2430 sec, Total Time: 3.6790 sec
Epsilon: 0.195, Bui

**--------Epsilon Opt Value For Rebuild------------**

In [ ]:
import numpy as np

# Epsilon sizes
epsilon_sizes = np.array(
   [0.15 ,0.155,0.16 ,0.165,0.17 ,0.175,0.18 ,0.185,0.19 ,0.195,0.2  ,0.205,
 0.21 ,0.215,0.22 ,0.225,0.23 ,0.235,0.24 ,0.245,0.25 ]
    )

# Total times from two runs (or versions)
total_times_1 = np.array(
   [4.0563,3.9737,3.8499,3.7772,3.8494,3.688 ,3.723 ,3.7787,3.6495,3.6958,
 3.5997,3.543 ,3.6427,3.59  ,3.6015,3.5572,3.5687,3.52  ,3.4939,3.5914,
 3.6014]
    )

total_times_2 = np.array(
   [3.9131,3.911 ,3.8   ,3.8531,3.8177,3.884 ,3.7872,3.6835,3.679 ,3.6527,
 3.6078,3.596 ,3.6821,3.596 ,3.607 ,3.699 ,3.5953,3.5584,3.5895,3.5025,
 3.5138]
    )

# Average total time across the two runs
avg_total_times = (total_times_1 + total_times_2) / 2

# Index of the minimum average total time
min_idx = np.argmin(avg_total_times)

# Corresponding leaf size
optimal_epsilon_size = epsilon_sizes[min_idx]

# Print results
print(f"Average total time (sec): {avg_total_times[min_idx]:.4f}")
print(f"Minimum found at epsilon size: {optimal_epsilon_size}")


Average total time (sec): 3.5392
Minimum found at epsilon size: 0.235


**------Node Optimization For Insert-----------**

**---------------Run 1-----------------------**

In [ ]:
import time
from tqdm import tqdm

# Test code for KD Tree
results = []

leaf_size_rebuild = 2050
epsilon_rebuild = 0.235

# Define leaf size range
leaf_size_range_start = 100
leaf_size_range_end = 2500
leaf_size_step = 100

data_size, query_size = test_combinations[0]

# Obtain Dataset Sample
current_data_rebuild = dataset[:MB]  # Directly slicing for clarity

# Retrieve the pre-generated queries from the list using the same order
queries = queries_list[0]  # Use first query set, as per your specification

for idx, leaf_size in enumerate(range(leaf_size_range_start, leaf_size_range_end + leaf_size_step, leaf_size_step)):
    print(f"\n{idx+1}: === Testing with Leaf Size: {leaf_size:,}, Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    # Build the tree with 100K points, using the optimized n_s, and epsilon
    kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
    kdtree.build(current_data_rebuild)

    # Obtain Dataset Sample
    current_data = dataset[MB:data_size+MB]
    # --- Build the KD Tree With Insertions---
    build_start = time.perf_counter()
    kdtree.leaf_size = leaf_size
    for pt in tqdm(current_data, desc="Inserting KD Tree"):
        kdtree.insert(pt)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying (Leaf Size {leaf_size:,})"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    search_time = search_end - search_start

    total_time = build_time + search_time
    results.append((leaf_size, build_time, search_time, total_time))

    print("-------------------------------------------------------------")
    print(f"Build Time: {build_time:.4f} sec, Search Time: {search_time:.4f} sec")
    print(f"Leaf Size: {leaf_size:,}, Total Time: {total_time:.4f} sec")

print("\n=== Summary of KD Tree Node Size Test ===")
for l_size, b_time, s_time, t_time in results:
    print(f"Leaf Size: {l_size:,}, Build Time: {b_time:.4f} sec, Search Time: {s_time:.4f} sec, Total Time: {t_time:.4f} sec")



1: === Testing with Leaf Size: 100, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Leaf Size 100): 100%|██████████| 20/20 [00:01<00:00, 15.81it/s]


-------------------------------------------------------------
Build Time: 5.7084 sec, Search Time: 1.4604 sec
Leaf Size: 100, Total Time: 7.1688 sec

2: === Testing with Leaf Size: 200, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Leaf Size 200): 100%|██████████| 20/20 [00:01<00:00, 16.09it/s]


-------------------------------------------------------------
Build Time: 5.7171 sec, Search Time: 1.4399 sec
Leaf Size: 200, Total Time: 7.1570 sec

3: === Testing with Leaf Size: 300, Data Size: 1,000,000 points, Query Size: 20 queries ===


...

-------------------------------------------------------------
Build Time: 5.7220 sec, Search Time: 1.4434 sec
Leaf Size: 2,300, Total Time: 7.1654 sec

24: === Testing with Leaf Size: 2,400, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Leaf Size 2,400): 100%|██████████| 20/20 [00:01<00:00, 15.63it/s]


-------------------------------------------------------------
Build Time: 5.6891 sec, Search Time: 1.4840 sec
Leaf Size: 2,400, Total Time: 7.1731 sec

25: === Testing with Leaf Size: 2,500, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Leaf Size 2,500): 100%|██████████| 20/20 [00:01<00:00, 15.78it/s]

-------------------------------------------------------------
Build Time: 5.5859 sec, Search Time: 1.4683 sec
Leaf Size: 2,500, Total Time: 7.0542 sec

=== Summary of KD Tree Node Size Test ===
Leaf Size: 100, Build Time: 5.7084 sec, Search Time: 1.4604 sec, Total Time: 7.1688 sec
Leaf Size: 200, Build Time: 5.7171 sec, Search Time: 1.4399 sec, Total Time: 7.1570 sec
Leaf Size: 300, Build Time: 5.6925 sec, Search Time: 1.4361 sec, Total Time: 7.1285 sec
Leaf Size: 400, Build Time: 5.7513 sec, Search Time: 1.4223 sec, Total Time: 7.1736 sec
Leaf Size: 500, Build Time: 5.6720 sec, Search Time: 1.4324 sec, Total Time: 7.1044 sec
Leaf Size: 600, Build Time: 5.7187 sec, Search Time: 1.4364 sec, Total Time: 7.1552 sec
Leaf Size: 700, Build Time: 5.6657 sec, Search Time: 1.4621 sec, Total Time: 7.1278 sec
Leaf Size: 800, Build Time: 5.6313 sec, Search Time: 1.4404 sec, Total Time: 7.0717 sec
Leaf Size: 900, Build Time: 5.6616 sec, Search Time: 1.4448 sec, Total Time: 7.1065 sec
Leaf Size: 1,0

**-------------Run 2-----------------------**

In [ ]:
import time
from tqdm import tqdm

# Test code for KD Tree
results = []

leaf_size_rebuild = 2050
epsilon_rebuild = 0.235

# Define leaf size range
leaf_size_range_start = 100
leaf_size_range_end = 2500
leaf_size_step = 100

data_size, query_size = test_combinations[0]

# Obtain Dataset Sample
current_data_rebuild = dataset[:MB]  # Directly slicing for clarity

# Retrieve the pre-generated queries from the list using the same order
queries = queries_list[0]  # Use first query set, as per your specification

for idx, leaf_size in enumerate(range(leaf_size_range_start, leaf_size_range_end + leaf_size_step, leaf_size_step)):
    print(f"\n{idx+1}: === Testing with Leaf Size: {leaf_size:,}, Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    # Build the tree with 100K points, using the optimized n_s, and epsilon
    kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
    kdtree.build(current_data_rebuild)

    # Obtain Dataset Sample
    current_data = dataset[MB:data_size+MB]
    # --- Build the KD Tree With Insertions---
    build_start = time.perf_counter()
    kdtree.leaf_size = leaf_size
    for pt in tqdm(current_data, desc="Inserting KD Tree"):
        kdtree.insert(pt)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying (Leaf Size {leaf_size:,})"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    search_time = search_end - search_start

    total_time = build_time + search_time
    results.append((leaf_size, build_time, search_time, total_time))

    print("-------------------------------------------------------------")
    print(f"Build Time: {build_time:.4f} sec, Search Time: {search_time:.4f} sec")
    print(f"Leaf Size: {leaf_size:,}, Total Time: {total_time:.4f} sec")

print("\n=== Summary of KD Tree Node Size Test ===")
for l_size, b_time, s_time, t_time in results:
    print(f"Leaf Size: {l_size:,}, Build Time: {b_time:.4f} sec, Search Time: {s_time:.4f} sec, Total Time: {t_time:.4f} sec")



1: === Testing with Leaf Size: 100, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Leaf Size 100): 100%|██████████| 20/20 [00:01<00:00, 15.81it/s]


-------------------------------------------------------------
Build Time: 5.6664 sec, Search Time: 1.4628 sec
Leaf Size: 100, Total Time: 7.1292 sec

2: === Testing with Leaf Size: 200, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Leaf Size 200): 100%|██████████| 20/20 [00:01<00:00, 16.02it/s]


-------------------------------------------------------------
Build Time: 5.7355 sec, Search Time: 1.4495 sec
Leaf Size: 200, Total Time: 7.1849 sec

3: === Testing with Leaf Size: 300, Data Size: 1,000,000 points, Query Size: 20 queries ===


...

-------------------------------------------------------------
Build Time: 5.6616 sec, Search Time: 1.4329 sec
Leaf Size: 2,300, Total Time: 7.0945 sec

24: === Testing with Leaf Size: 2,400, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Leaf Size 2,400): 100%|██████████| 20/20 [00:01<00:00, 16.08it/s]


-------------------------------------------------------------
Build Time: 5.7813 sec, Search Time: 1.4415 sec
Leaf Size: 2,400, Total Time: 7.2228 sec

25: === Testing with Leaf Size: 2,500, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Leaf Size 2,500): 100%|██████████| 20/20 [00:01<00:00, 16.10it/s]

-------------------------------------------------------------
Build Time: 5.8416 sec, Search Time: 1.4455 sec
Leaf Size: 2,500, Total Time: 7.2870 sec

=== Summary of KD Tree Node Size Test ===
Leaf Size: 100, Build Time: 5.6664 sec, Search Time: 1.4628 sec, Total Time: 7.1292 sec
Leaf Size: 200, Build Time: 5.7355 sec, Search Time: 1.4495 sec, Total Time: 7.1849 sec
Leaf Size: 300, Build Time: 5.6764 sec, Search Time: 1.4493 sec, Total Time: 7.1257 sec
Leaf Size: 400, Build Time: 5.7189 sec, Search Time: 1.4549 sec, Total Time: 7.1738 sec
Leaf Size: 500, Build Time: 5.6452 sec, Search Time: 1.4512 sec, Total Time: 7.0964 sec
Leaf Size: 600, Build Time: 5.8545 sec, Search Time: 1.4467 sec, Total Time: 7.3013 sec
Leaf Size: 700, Build Time: 5.6611 sec, Search Time: 1.4612 sec, Total Time: 7.1223 sec
Leaf Size: 800, Build Time: 5.5854 sec, Search Time: 1.4541 sec, Total Time: 7.0395 sec
Leaf Size: 900, Build Time: 5.7569 sec, Search Time: 1.4619 sec, Total Time: 7.2189 sec
Leaf Size: 1,0

**----Node Size Optimization For Insertion---------**

In [ ]:
import numpy as np

# Leaf sizes
leaf_sizes = np.array(
    [ 100, 200, 300, 400, 500, 600, 700, 800, 900,1000,1100,1200,1300,1400,
 1500,1600,1700,1800,1900,2000,2100,2200,2300,2400,2500]
    )

# Total times from two runs (or versions)
total_times_1 = np.array(
     [7.1688,7.157 ,7.1285,7.1736,7.1044,7.1552,7.1278,7.0717,7.1065,7.1248,
 7.1882,7.2795,7.3055,7.0791,7.2668,7.1138,7.372 ,7.1454,7.0526,7.3196,
 7.0416,7.3015,7.1654,7.1731,7.0542]
    )

total_times_2 = np.array(
    [7.1292,7.1849,7.1257,7.1738,7.0964,7.3013,7.1223,7.0395,7.2189,7.0521,
 7.0721,7.2394,7.0834,7.0579,7.0406,7.1058,7.2672,7.0251,7.1477,7.0614,
 7.0596,7.0143,7.0945,7.2228,7.287 ]
    )

# Average total time across the two runs
avg_total_times = (total_times_1 + total_times_2) / 2

# Index of the minimum average total time
min_idx = np.argmin(avg_total_times)

# Corresponding leaf size
optimal_leaf_size = leaf_sizes[min_idx]

# Print results
print(f"Average total time (sec): {avg_total_times[min_idx]:.4f}")
print(f"Minimum found at leaf size: {optimal_leaf_size}")


Average total time (sec): 7.0506
Minimum found at leaf size: 2100


**------------Epsilon Optimization for Insertion-------------**

**-------------------Run 1---------------------------**

In [17]:
import time
from tqdm import tqdm

# Test code for KD Tree
results = []

leaf_size_rebuild = 2050
epsilon_rebuild = 0.235

leaf_size_insert = 2100

# Define epsilon range
epsilon_range_start = 0.0
epsilon_range_end = 0.07
epsilon_size_step = 0.005

# Obtain Dataset Sample
current_data_rebuild = dataset[:MB]  # Directly slicing for clarity

# Retrieve the pre-generated queries from the list using the same order
queries = queries_list[0]  # Use first query set, as per your specification

for idx, epsilon in enumerate(np.arange(epsilon_range_start, epsilon_range_end + epsilon_size_step, epsilon_size_step)):
    epsilon = round(epsilon, 10)
    print(f"\n{idx+1}: === Testing with Epsilon: {epsilon:,}, Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    # Build the tree with 100K points, using the optimized n_s, and epsilon
    kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
    kdtree.build(current_data_rebuild)

    # Readjust for the insertion
    data_size, query_size = test_combinations[0]
    # Obtain Dataset Sample
    current_data = dataset[MB:data_size+MB]

    kdtree.leaf_size = leaf_size_insert
    kdtree.epsilon = epsilon

    # --- Build the KD Tree With Insertions---
    build_start = time.perf_counter()
    for pt in tqdm(current_data, desc="Inserting KD Tree"):
        kdtree.insert(pt)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying (Epsilon Size {epsilon:,})"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    search_time = search_end - search_start

    total_time = build_time + search_time
    results.append((epsilon, build_time, search_time, total_time))

    print("-------------------------------------------------------------")
    print(f"Build Time: {build_time:.4f} sec, Search Time: {search_time:.4f} sec")
    print(f"Epsilon: {epsilon:,}, Total Time: {total_time:.4f} sec")

print("\n=== Summary of KD Tree Epsilon Size Test ===")
for e_size, b_time, s_time, t_time in results:
    print(f"Epsilon: {e_size:,}, Build Time: {b_time:.4f} sec, Search Time: {s_time:.4f} sec, Total Time: {t_time:.4f} sec")



1: === Testing with Epsilon: 0.0, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon Size 0.0): 100%|██████████| 20/20 [00:02<00:00,  9.70it/s]


-------------------------------------------------------------
Build Time: 8.2880 sec, Search Time: 2.3961 sec
Epsilon: 0.0, Total Time: 10.6841 sec

2: === Testing with Epsilon: 0.005, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon Size 0.005): 100%|██████████| 20/20 [00:01<00:00, 10.11it/s]


-------------------------------------------------------------
Build Time: 8.1564 sec, Search Time: 2.3048 sec
Epsilon: 0.005, Total Time: 10.4612 sec

3: === Testing with Epsilon: 0.01, Data Size: 1,000,000 points, Query Size: 20 queries ===


...

-------------------------------------------------------------
Build Time: 8.0005 sec, Search Time: 2.3261 sec
Epsilon: 0.065, Total Time: 10.3266 sec

15: === Testing with Epsilon: 0.07, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon Size 0.07): 100%|██████████| 20/20 [00:01<00:00, 10.23it/s]


-------------------------------------------------------------
Build Time: 7.9025 sec, Search Time: 2.2735 sec
Epsilon: 0.07, Total Time: 10.1760 sec

16: === Testing with Epsilon: 0.075, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon Size 0.075): 100%|██████████| 20/20 [00:01<00:00, 10.17it/s]

-------------------------------------------------------------
Build Time: 8.0175 sec, Search Time: 2.2902 sec
Epsilon: 0.075, Total Time: 10.3077 sec

=== Summary of KD Tree Epsilon Size Test ===
Epsilon: 0.0, Build Time: 8.2880 sec, Search Time: 2.3961 sec, Total Time: 10.6841 sec
Epsilon: 0.005, Build Time: 8.1564 sec, Search Time: 2.3048 sec, Total Time: 10.4612 sec
Epsilon: 0.01, Build Time: 8.0233 sec, Search Time: 2.2889 sec, Total Time: 10.3122 sec
Epsilon: 0.015, Build Time: 8.0914 sec, Search Time: 2.3235 sec, Total Time: 10.4149 sec
Epsilon: 0.02, Build Time: 8.0257 sec, Search Time: 2.3014 sec, Total Time: 10.3271 sec
Epsilon: 0.025, Build Time: 7.9349 sec, Search Time: 2.3563 sec, Total Time: 10.2912 sec
Epsilon: 0.03, Build Time: 7.9172 sec, Search Time: 2.3182 sec, Total Time: 10.2354 sec
Epsilon: 0.035, Build Time: 8.2231 sec, Search Time: 2.3489 sec, Total Time: 10.5720 sec
Epsilon: 0.04, Build Time: 8.1164 sec, Search Time: 2.3608 sec, Total Time: 10.4773 sec
Epsilon: 

**----------------Run 2-------------------------**

In [15]:
import time
from tqdm import tqdm

# Test code for KD Tree
results = []

leaf_size_rebuild = 2050
epsilon_rebuild = 0.235

leaf_size_insert = 2100

# Define epsilon range
epsilon_range_start = 0.0
epsilon_range_end = 0.07
epsilon_size_step = 0.005

# Obtain Dataset Sample
current_data_rebuild = dataset[:MB]  # Directly slicing for clarity

# Retrieve the pre-generated queries from the list using the same order
queries = queries_list[0]  # Use first query set, as per your specification

for idx, epsilon in enumerate(np.arange(epsilon_range_start, epsilon_range_end + epsilon_size_step, epsilon_size_step)):
    epsilon = round(epsilon, 10)
    print(f"\n{idx+1}: === Testing with Epsilon: {epsilon:,}, Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    # Build the tree with 100K points, using the optimized n_s, and epsilon
    kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
    kdtree.build(current_data_rebuild)

    # Readjust for the insertion
    data_size, query_size = test_combinations[0]
    # Obtain Dataset Sample
    current_data = dataset[MB:data_size+MB]

    kdtree.leaf_size = leaf_size_insert
    kdtree.epsilon = epsilon

    # --- Build the KD Tree With Insertions---
    build_start = time.perf_counter()
    for pt in tqdm(current_data, desc="Inserting KD Tree"):
        kdtree.insert(pt)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying (Epsilon Size {epsilon:,})"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    search_time = search_end - search_start

    total_time = build_time + search_time
    results.append((epsilon, build_time, search_time, total_time))

    print("-------------------------------------------------------------")
    print(f"Build Time: {build_time:.4f} sec, Search Time: {search_time:.4f} sec")
    print(f"Epsilon: {epsilon:,}, Total Time: {total_time:.4f} sec")

print("\n=== Summary of KD Tree Epsilon Size Test ===")
for e_size, b_time, s_time, t_time in results:
    print(f"Epsilon: {e_size:,}, Build Time: {b_time:.4f} sec, Search Time: {s_time:.4f} sec, Total Time: {t_time:.4f} sec")



1: === Testing with Epsilon: 0.0, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon Size 0.0): 100%|██████████| 20/20 [00:02<00:00,  9.61it/s]


-------------------------------------------------------------
Build Time: 8.1512 sec, Search Time: 2.4034 sec
Epsilon: 0.0, Total Time: 10.5547 sec

2: === Testing with Epsilon: 0.005, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon Size 0.005): 100%|██████████| 20/20 [00:01<00:00, 10.11it/s]


-------------------------------------------------------------
Build Time: 8.1657 sec, Search Time: 2.3068 sec
Epsilon: 0.005, Total Time: 10.4725 sec

3: === Testing with Epsilon: 0.01, Data Size: 1,000,000 points, Query Size: 20 queries ===


...

-------------------------------------------------------------
Build Time: 8.1832 sec, Search Time: 2.3175 sec
Epsilon: 0.065, Total Time: 10.5007 sec

15: === Testing with Epsilon: 0.07, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon Size 0.07): 100%|██████████| 20/20 [00:01<00:00, 10.22it/s]


-------------------------------------------------------------
Build Time: 8.1996 sec, Search Time: 2.2788 sec
Epsilon: 0.07, Total Time: 10.4784 sec

16: === Testing with Epsilon: 0.075, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon Size 0.075): 100%|██████████| 20/20 [00:02<00:00,  9.79it/s]

-------------------------------------------------------------
Build Time: 8.1215 sec, Search Time: 2.3920 sec
Epsilon: 0.075, Total Time: 10.5135 sec

=== Summary of KD Tree Epsilon Size Test ===
Epsilon: 0.0, Build Time: 8.1512 sec, Search Time: 2.4034 sec, Total Time: 10.5547 sec
Epsilon: 0.005, Build Time: 8.1657 sec, Search Time: 2.3068 sec, Total Time: 10.4725 sec
Epsilon: 0.01, Build Time: 8.2796 sec, Search Time: 2.3729 sec, Total Time: 10.6525 sec
Epsilon: 0.015, Build Time: 8.2770 sec, Search Time: 2.3313 sec, Total Time: 10.6082 sec
Epsilon: 0.02, Build Time: 8.2759 sec, Search Time: 2.2978 sec, Total Time: 10.5737 sec
Epsilon: 0.025, Build Time: 8.2470 sec, Search Time: 2.3639 sec, Total Time: 10.6109 sec
Epsilon: 0.03, Build Time: 8.1997 sec, Search Time: 2.4039 sec, Total Time: 10.6036 sec
Epsilon: 0.035, Build Time: 8.2297 sec, Search Time: 2.5777 sec, Total Time: 10.8074 sec
Epsilon: 0.04, Build Time: 8.4741 sec, Search Time: 2.4932 sec, Total Time: 10.9673 sec
Epsilon: 

**------Epsilon Optimization For Insertion----**

In [18]:
import numpy as np

# Epsilon sizes
epsilon_sizes = np.array(
    [0.   ,0.005,0.01 ,0.015,0.02 ,0.025,0.03 ,0.035,0.04 ,0.045,0.05 ,0.055,
 0.06 ,0.065,0.07 ,0.075]
)

# Total times from two runs (or versions)
total_times_1 = np.array(
    [10.6841,10.4612,10.3122,10.4149,10.3271,10.2912,10.2354,10.572 ,10.4773,
 10.3688,10.2023,10.3414,10.3672,10.3266,10.176 ,10.3077]
)

total_times_2 = np.array(
     [10.5547,10.4725,10.6525,10.6082,10.5737,10.6109,10.6036,10.8074,10.9673,
 10.4835,10.6569,10.521 ,10.3274,10.5007,10.4784,10.5135]
)

# Average total time across the two runs
avg_total_times = (total_times_1 + total_times_2) / 2

# Index of the minimum average total time
min_idx = np.argmin(avg_total_times)

# Corresponding leaf size
optimal_epsilon_size = epsilon_sizes[min_idx]

# Print results
print(f"Average total time (sec): {avg_total_times[min_idx]:.4f}")
print(f"Minimum found at epsilon size: {optimal_epsilon_size}")


Average total time (sec): 10.3272
Minimum found at epsilon size: 0.07
